# Step 2: Assigning Speed to Traffic Data

## These series of notebooks rely on the 'load-traffic' branch of the GOSTnets repo and has not yet been merged into master. The link to the code is here: https://github.com/worldbank/GOSTnets/tree/load_traffic

In [ ]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

from shapely.geometry import LineString, Point

In [ ]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

In [ ]:
# read graph
G = nx.read_gpickle('../mapbox_traffic/sri_lanka_unclean2.pickle')

In [ ]:
len(G.edges)

In [ ]:
gn.example_edge(G, 5)

### Optional step: Find out average speeds for highway classes with Mapbox data
This function below will calculate the average speeds for all highways that have traffic information per highway classes

In [ ]:
def find_traffic_hwy_avg_speeds_by_class(G, speed_tag='mean_speed'):
    """
    Function for finding out the different highway classes in the graph and their respective lengths

    :param G: a graph object
    :param speed_tag: specifies which edge attribute represents traffic speed
    :returns: a dictionary that has each class and the total distance per class
    """

    if type(G) == nx.classes.multidigraph.MultiDiGraph or type(G) == nx.classes.digraph.DiGraph:
        pass
    else:
        raise ValueError('Expecting a graph or geodataframe for G!')

    G_adj = G.copy()

    traffic_class_list = []

    for u, v, data in G_adj.edges(data=True):
        #print(data['infra_type'])
        if 'mean_speed' in data:
            if type(data['infra_type']) == list:
                    if data['infra_type'][0] not in traffic_class_list:
                        traffic_class_list.append(data['infra_type'][0])
            else:
                if data['infra_type'] not in traffic_class_list:
                    traffic_class_list.append(data['infra_type'])
    
    class_dict = { i : [] for i in traffic_class_list }
    
    print(f'print traffic_class_list: {traffic_class_list}')

    for i in traffic_class_list:
        for u, v, data in G_adj.edges(data=True):
            if 'mean_speed' in data:
                if data['mean_speed'] > 0:
                    if type(data['infra_type']) == list:
                        if data['infra_type'][0] == i:
                            class_dict[i].append(data[speed_tag])
                    else:
                        if data['infra_type'] == i:
                            class_dict[i].append(data[speed_tag])
                        
    print(f'print class_dict: {class_dict}')
    
    def Average(lst): 
        if len(lst) > 0:
            return sum(lst) / len(lst)
        else:
            return 0
    
    class_dict_copy = class_dict.copy()
    
    for key in class_dict_copy:
        class_dict_copy[key] = Average(class_dict_copy[key])

    return class_dict_copy

In [ ]:
average_speeds_per_class_dict = find_traffic_hwy_avg_speeds_by_class(G, speed_tag='mean_speed')

In [ ]:
average_speeds_per_class_dict

## Speed Dictionaries
The mapbox speed dictionary was generated using the function above. The max speeds dictionary was generated by opening the edges in ArcGIS. It is up to you which dictionary to use in your analysis.

In [ ]:
# speed dict based on mean of mapbox real-world speeds per class
speed_dict_sri_lanka_mapbox_mean_speeds = {
'motorway':85 ,
'motorway_link':30 ,
'trunk': 40,
'trunk_link': 23,
'primary': 35,
'primary_link': 14,
'secondary': 34,
'secondary_link': 9,
'tertiary':25,
'tertiary_link': 13,
'residential': 20,
'unclassified': 20,
'track': 20,
'service': 10
}

In [ ]:
# speed dict based on existing speed limit tags
speed_dict_sri_lanka_max_speeds = {
'motorway':100 ,
'motorway_link':35 ,
'trunk': 60,
'trunk_link': 50,
'primary': 60,
'primary_link': 50,
'secondary': 50,
'secondary_link': 45,
'tertiary':40,
'tertiary_link': 40,
'residential': 30,
'unclassified': 25,
'track': 25,
'service': 20
}

## Function to add time to edges
The function below was based off of a GOSTnets function. It was modified to calculate the correct time based on whether a Mapbox speed exists or not. Also, it creates a new 'speed' attribute that is either based on the mapbox traffic speed, or the speed provided by the input dictionary if the mapbox traffic speed does not exist.

In [ ]:
def convert_network_to_time_w_traffic(G, distance_tag, graph_type = 'drive', road_col = 'highway', traffic_col = 'mean_speed', speed_dict = speed_dict_sri_lanka_max_speeds, walk_speed = 4.5, factor = 1, default = None, improvement_cost_per_km = None):
    """
    Function for adding a time value to edge dictionaries. Ensure any GeoDataFrames / graphs are in the same projection before using function, or pass a crs.

    DEFAULT SPEEDS:

               speed_dict = {
               'residential': 20,  # kmph
               'primary': 40, # kmph
               'primary_link':35,
               'motorway':50,
               'motorway_link': 45,
               'trunk': 40,
               'trunk_link':35,
               'secondary': 30,
               'secondary_link':25,
               'tertiary':30,
               'tertiary_link': 25,
               'unclassified':20
               }

    :param G: a graph containing one or more nodes
    :param distance_tag: the key in the dictionary for the field currently
               containing a distance in meters
    :param road_col: key for the road type in the edge data dictionary
    :param graph_type: set to either 'drive' or 'walk'. IF walk - will set time = walking time across all segment, using the supplied walk_speed. IF drive - will use a speed dictionary for each road type, or defaults as per the note below.
    :param speed_dict: speed dictionary to use. If not supplied, reverts to
               defaults
    :param walk_speed: specify a walkspeed in km/h
    :param factor: allows you to scale up / down distances if saved in a unit other than meters. Set to 1000 if length in km.
    :param default: if highway type not in the speed_dict, use this road class as an in-fill value for time.
    :returns: The original graph with a new data property for the edges called 'time'
    """

    if type(G) == nx.classes.multidigraph.MultiDiGraph or type(G) == nx.classes.digraph.DiGraph:
        pass
    else:
        raise ValueError('Expecting a graph or geodataframe for G!')

    import warnings

    try:
        # checks the first edge to see if the 'time' attribute already exists
        if list(G.edges(data = True))[0][2]['time']:
          warnings.warn('Aree you sure you want to convert length to time? This graph already has a time attribute')
    except:
        pass

    G_adj = G.copy()

    for u, v, data in G_adj.edges(data=True):

        # the default lenth in the graph should be in km
        orig_len = data[distance_tag]

        # Note that this is a MultiDiGraph so there could
        # be multiple indices here, I naively assume this is not
        # the case
        data['length'] = orig_len * factor
        

        # get appropriate speed limit
        if graph_type == 'walk':
            speed = walk_speed

        elif graph_type == 'drive':

            if speed_dict == None:
                speed_dict = {
                'residential': 20,  # kmph
                'primary': 40, # kmph
                'primary_link':35,
                'motorway':50,
                'motorway_link': 45,
                'trunk': 40,
                'trunk_link':35,
                'secondary': 30,
                'secondary_link':25,
                'tertiary':30,
                'tertiary_link': 25,
                'unclassified':20
                }

            highwayclass = data[road_col]
            
            trafficclass = data[traffic_col]

            if trafficclass > 0:
                speed = data[traffic_col]
                data['speed'] = speed
            else:
                if type(highwayclass) == list:
                    highwayclass = highwayclass[0]

                if highwayclass in speed_dict.keys():
                    speed = speed_dict[highwayclass]
                else:
                    if default == None:
                        speed = 20
                    else:
                        speed = speed_dict[default]
                data['speed'] = speed
            
            #calculate road improvement costs for secondary or lower highways
            if highwayclass in ['secondary', 'secondary_link', 'tertiary', 'tertiary_link', 'unclassified', 'residential']:
                data['imp_cost'] = orig_len * improvement_cost_per_km

        else:
            raise ValueError('Expecting either a graph_type of "walk" or "drive"!')

        # perform conversion
        hours = orig_len / speed
        in_seconds = hours * 60 * 60
        data['time'] = in_seconds

        # And state the mode, too
        data['mode'] = graph_type

    return G_adj

In [ ]:
G_time = convert_network_to_time_w_traffic(G, distance_tag = 'length', road_col = 'infra_type', speed_dict = speed_dict_sri_lanka_max_speeds, factor = 1, improvement_cost_per_km = 186800)

In [ ]:
gn.example_edge(G_time, 15)

In [ ]:
#save graph again
#gn.save(G_time,'sri_lanka_unclean2_w_time','./', pickle = True, edges = True, nodes = True)

## Export edges as shapefile to visualize

In [ ]:
edge_gdf_w_traffic = gn.edge_gdf_from_graph(G_time)

In [ ]:
edge_gdf_w_traffic.sort_values(by=['speed'])

In [ ]:
#edge_gdf_w_traffic.to_file(driver = 'ESRI Shapefile', filename = './sri_lanka_hwy_w_traffic_and_time2.shp')

### now take the biggest sub-graph and compare

In [ ]:
# before
# let's print info on our clean version
print(nx.info(G_time))

In [ ]:
# Identify only the largest graph

# compatible with NetworkX 2.4
list_of_subgraphs = list(G_time.subgraph(c).copy() for c in nx.strongly_connected_components(G_time))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i

# set your graph equal to the largest sub-graph
G_largest = max_graph

In [ ]:
# print info about the largest sub-graph
print(nx.info(G_largest))

In [ ]:
# re-save
#edge_gdf_w_traffic_largest = gn.edge_gdf_from_graph(G_largest)

In [ ]:
#edge_gdf_w_traffic_largest.to_file(driver = 'ESRI Shapefile', filename = './sri_lanka_hwy_w_traffic_and_time2_largest.shp')

## Save your graph

In [ ]:
#save graph again
gn.save(G_largest,'./mapbox_traffic/sri_lanka_unclean2_w_time_largest_20200616_max_speeds','./', pickle = True, edges = False, nodes = False)